In [5]:
import os
from mongo_connect import load_client
import pandas as pd 
from pandas_gbq import to_gbq
import google.auth

client = load_client()
CURR_DIR = os.getcwd()

db = client.get_database('Cluster0')
col = db.get_collection('games_rating')





In [9]:
import datetime
import calendar
import pendulum 


custom_date = pendulum.datetime(2015,1,1,tz='America/Toronto')
print(type(custom_date))
def six_mounth_ago(custom_date): 

    curr_month = custom_date.month
    curr_day = custom_date.day

    months = []
    for i in range(7): 
        months.append(curr_month)
        if curr_month != 1:
            curr_month -= 1
        else:
            curr_month = 12

    day_count = []
    for i in months: 
        num_days = calendar.monthrange(custom_date.year, i)[1]
        day_count.append(num_days)

    last_year = months[0] < months[-1]

    if last_year: 
        six_mounth_ago = datetime.datetime(custom_date.year - 1, months[-1], custom_date.day)
    else:
        six_mounth_ago = datetime.datetime(custom_date.year, months[-1], custom_date.day)

    print(six_mounth_ago)
    six_mounth_ago_unix = six_mounth_ago.timestamp()

    return six_mounth_ago_unix

six_mounth_ago_unix = six_mounth_ago(custom_date)



<class 'pendulum.datetime.DateTime'>
2014-07-01 00:00:00


In [18]:
print(six_mounth_ago_unix)
result = col.find({'unixReviewTime': {'$gt': six_mounth_ago_unix}})

documents = list(result)
df = pd.DataFrame(documents)

print(datetime.datetime.fromtimestamp(1536278400))


1404165600.0
2018-09-07 02:00:00


In [ ]:
# df = df.drop(columns=['image', 'style', 'reviewText', 'summary'])

datetime.datetime.fromtimestamp(df['unixReviewTime'].head(1)[0])


In [ ]:
df['average_overall'] = df.groupby(by='asin')['overall'] \
            .transform('mean')

# find 15 best games 
df_best_15 = df.sort_values(by='average_overall', ascending=False).drop_duplicates('asin').head(15)


df_best_15.head(5)

In [4]:
df['game_id'] = df['asin'] 
df['avg_note'] = df['average_overall']
df['user_note'] = df['overall']
df['real_date'] = df['unixReviewTime'].apply(datetime.datetime.fromtimestamp)
df['latest_note'] = df.groupby('asin')['unixReviewTime'].transform('max').astype(int)
df['oldest_note'] = df.groupby('asin')['unixReviewTime'].transform('min').astype(int)

to_drop = [col for col in df.columns if col not in ['game_id', 'avg_note', 'user_note', 'latest_note', 'oldest_note', 'real_date']]

print(to_drop)
df.drop(columns=to_drop, inplace=True)




NameError: name 'df' is not defined

In [ ]:
df.tail()

In [ ]:

"""
df['latest_note_date'] = pd.to_datetime(df['latest_note']).dt.date
df['oldest_note_date'] = pd.to_datetime(df['oldest_note']).dt.date
df.head()
"""

In [ ]:
df['user_note'] = df['user_note'].astype(int)
df['game_id'] = (df['game_id'].astype('category').cat.codes + 1).astype(int)

df['latest_note'] = df['latest_note'].apply(lambda x: pd.to_datetime(x))
df['oldest_note'] = df['oldest_note'].apply(lambda x: pd.to_datetime(x))

df['latest_note_sql'] = df['latest_note'].dt.strftime('%Y-%m-%d')
df['oldest_note_sql'] = df['oldest_note'].dt.strftime('%Y-%m-%d')

df.head()

In [ ]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "google_credentials.json"

credentials, project_id = google.auth.default()

to_gbq(df, 'dengineer-413113.datalake.video_games', project_id='dengineer-413113', credentials=credentials)